## Starting with the Velocity Humanizer — proposed interfaces and prompt
Below are lightweight, plug‑in snippets you can drop into src/graphs/clip_graph.py (or a small helper module) to add a velocity humanization pass between generation and storage.
Minimal SML dict assumptions (read/write)
•
Input SML clip dict (same structure clip_graph.generate_sml_clip already returns):
◦
{"clip_id": Optional[int], "name": str, "track_name": str, "bars": List[Bar]}
◦
Bar = {"bar_index": int, "items": List[Item], "expression": Optional[dict]}
◦
Item for notes: {"note": "C4", "duration": "quarter", "velocity": Optional[int]=90, ...}
◦
Item for rests: {"rest": "eighth"} (no velocity)
•
Output: same dict shape; only note velocity fields are modified in place. No structural changes.

## Python: Humanizer tool signature (pure function)

In [ ]:
from __future__ import annotations
from typing import Any, Dict, List, Optional, TypedDict, Literal
import random

from langgraph.graph import StateGraph

DEFAULT_UNITS_PER_BAR = 32  # import from src.dsl.sml_ast if available

class HumanizeParams(TypedDict, total=False):
    # Overall approach
    strategy: Literal["meter_accent", "flat+jitter", "crescendo", "diminuendo"]
    meter: Literal["4/4", "3/4", "6/8"]
    intensity: float  # 0.0..1.0 master scale for all effects

    # Base and limits
    base_velocity: int  # starting nominal velocity (e.g., 90)
    min_velocity: int   # clamp lower bound (e.g., 20)
    max_velocity: int   # clamp upper bound (e.g., 127)

    # Accents & jitter
    accent_gain_strong: int  # e.g., +12 for beat 1
    accent_gain_medium: int  # e.g., +6 for beat 3 (in 4/4)
    accent_gain_weak: int    # e.g., +2 for beats 2 & 4
    offbeat_jitter: int      # e.g., ±6 around offbeats
    random_jitter: int       # e.g., ±4 small randomness per note

    # Swing (if you later want to modulate timing-related accenting)
    swing: float             # 0.0..1.0; velocity nudge could mirror timing swing

    # Scope & determinism
    apply_to_bars: Optional[List[int]]  # 1-based bar indices to include; None = all
    seed: Optional[int]                 # RNG seed for reproducibility


def humanize_velocities(
    sml_clip: Dict[str, Any],
    params: Optional[HumanizeParams] = None,
) -> Dict[str, Any]:
    """Return a new SML clip dict with note velocities humanized.

    Contract:
    - Input: SML dict with bars/items; we read note items with a `velocity` int.
    - Output: Same shape; only note velocities are updated and clamped to 0..127.
    - Rests are not touched. Items lacking `velocity` will get one assigned.

    This is a pure function: it does not perform any DB calls and mutates
    a copy of the input dict.

    Suggested baseline behavior (4/4):
    - Beat 1: strong accent (+accent_gain_strong)
    - Beat 3: medium accent (+accent_gain_medium)
    - Beats 2 & 4: weak accent (+accent_gain_weak)
    - Offbeats get ±offbeat_jitter; every note gets small ±random_jitter
    - Apply overall `intensity` multiplier to all deltas
    - Clamp to [min_velocity, max_velocity]
    """
    # ...implementation to be filled; keep this signature stable
    return sml_clip

## Python: LangGraph node wrapper signature

In [ ]:
from typing import Any, Dict

# Assuming ClipGenerationState already includes `prompt`, `sml_clip`, `error`, etc.
# Add `humanize_params` (optional) to your state type.

async def apply_velocity_humanization(state: ClipGenerationState) -> ClipGenerationState:
    """If state has `sml_clip`, apply velocity humanization with optional
    `humanize_params`, then return updated state. If no params are provided,
    use safe defaults.
    """
    if state.get("error"):
        return state

    sml = state.get("sml_clip")
    if not sml:
        return {**state, "error": "No sml_clip in state for velocity humanization"}

    params: HumanizeParams = state.get("humanize_params", {})  # type: ignore
    try:
        updated = humanize_velocities(sml, params)
        return {**state, "sml_clip": updated}
    except Exception as e:
        return {**state, "error": f"Velocity humanization failed: {e}"}

State

In [ ]:
class ClipGenerationState(TypedDict, total=False):
    prompt: str
    sml_clip: Dict[str, Any]
    clip_id: int
    error: str
    humanize_params: HumanizeParams  # optional controls for velocity humanizer

Wiring the graph

In [2]:
# wiring
# Register the node
graph_builder = StateGraph(ClipGenerationState)
graph_builder.add_node("apply_velocity_humanization", apply_velocity_humanization)

# Place it between generation and storage
graph_builder.set_entry_point("generate_sml_clip")
graph_builder.add_edge("generate_sml_clip", "apply_velocity_humanization")
graph_builder.add_edge("apply_velocity_humanization", "store_clip")
graph_builder.add_edge("store_clip", END)

## OpenAI function schema (optional) for Expression Agent
If you want the Expression Agent to decide the parameters, expose a tool schema to your LLM:

In [ ]:
HUMANIZE_VELOCITIES_SCHEMA = {
    "name": "humanize_velocities_config",
    "description": "Propose parameters for velocity humanization given style/meter/goals.",
    "parameters": {
        "type": "object",
        "properties": {
            "strategy": {"type": "string", "enum": ["meter_accent", "flat+jitter", "crescendo", "diminuendo"]},
            "meter": {"type": "string", "enum": ["4/4", "3/4", "6/8"]},
            "intensity": {"type": "number", "minimum": 0.0, "maximum": 1.0},
            "base_velocity": {"type": "integer", "minimum": 0, "maximum": 127},
            "min_velocity": {"type": "integer", "minimum": 0, "maximum": 127},
            "max_velocity": {"type": "integer", "minimum": 0, "maximum": 127},
            "accent_gain_strong": {"type": "integer", "minimum": -64, "maximum": 64},
            "accent_gain_medium": {"type": "integer", "minimum": -64, "maximum": 64},
            "accent_gain_weak": {"type": "integer", "minimum": -64, "maximum": 64},
            "offbeat_jitter": {"type": "integer", "minimum": 0, "maximum": 32},
            "random_jitter": {"type": "integer", "minimum": 0, "maximum": 32},
            "swing": {"type": "number", "minimum": 0.0, "maximum": 1.0},
            "apply_to_bars": {"type": "array", "items": {"type": "integer", "minimum": 1}},
            "seed": {"type": "integer"}
        },
        "required": ["strategy", "meter", "intensity"]
    }
}

## Expression Agent prompt (sample)
Use this to elicit good defaults and styles from short English instructions (e.g., “gentle lofi swing, 4/4, keep it soft”).

You are an expression designer for MIDI performance. Given a short style request and the clip’s meter,
propose a JSON config for velocity humanization.

Goals:
- Preserve musicality and headroom. Never exceed velocity 127 or drop below 0.
- Strong beat accents should be tasteful; offbeats can get subtle jitter.
- Intensity controls the scale of all effects (0.0 = minimal, 1.0 = strong).
- Prefer 4/4 defaults if meter not specified by the user.

Return only tool arguments for `humanize_velocities_config`.

Guidelines:
- 4/4: Beat 1 strongest, Beat 3 medium, Beats 2/4 weak accents.
- Jitter is symmetric ±N and kept small (e.g., 2–6) unless intensity is high.
- Base velocity stays in 70–100 for most pop/lofi; clamp to [20, 120] unless user requests otherwise.
- Use a fixed seed for reproducibility when the user requests a deterministic preview.

User request: "{style_request}"
Meter: "{meter}"

Example of using the node in main()

In [ ]:
async def main() -> None:
    prompt = "Create a 2-bar ascending arpeggio in C major, quarter notes, ending with a rest."
    humanize_params: HumanizeParams = {
        "strategy": "meter_accent",
        "meter": "4/4",
        "intensity": 0.5,
        "base_velocity": 88,
        "min_velocity": 20,
        "max_velocity": 120,
        "accent_gain_strong": 12,
        "accent_gain_medium": 6,
        "accent_gain_weak": 2,
        "offbeat_jitter": 4,
        "random_jitter": 3,
        "seed": 42,
    }
    state: ClipGenerationState = {"prompt": prompt, "humanize_params": humanize_params}
    result = await clip_graph.ainvoke(state)
    print(result.get("clip_id"), result.get("error"))